In [ ]:
from pyprojroot import here
from langchain_community.utilities import SQLDatabase
# db_path="notebook\tools\sql tools\BIRD (1).db"
# db_path_root=here(db_path)
# db=SQLDatabase.from_uri(f"sqlite:///{db_path_root}")
sqldb_directory_path="BIRD (1).db"

# db=SQLDatabase.from_uri("sqlite:///{sqldb_directory_path}")
db=SQLDatabase.from_uri(f"sqlite:///{sqldb_directory_path}")



In [7]:
result=db.run("SELECT * FROM 'Task' LIMIT 0,30")
print(result)

[('T-001', 27, 'Interior Fit-Out', '1/23/2020', '2/22/2021', 0, 121235.8, 42939.82, 121235.8, 'CCo-03'), ('T-002', 3, 'Foundation Construction', '11/14/2017', '4/16/2020', 1, 566066.2, 602875.8, 645680.9, 'CCo-05'), ('T-003', 37, 'Site Survey', '1/27/2022', '1/27/2022', 1, 575075.2, 575075.2, 507631.2, 'CCo-09'), ('T-004', 47, 'Foundation Work', '3/26/2022', '3/26/2022', 1, 805677.1, 885158.3, 805677.1, 'CCo-01'), ('T-005', 8, 'Railings Installation', '12/15/2017', '9/14/2018', 0, 855508.8, 835542.4, 855508.8, 'CCo-01'), ('T-006', 1, 'Foundation Work', '2019-02-10 00:00:00', '2022-06-01 00:00:00', 1, 439495.0, 393913.8, 530709.1, 'CCo-05'), ('T-007', 26, 'Excavation', '11/23/2021', '2024-08-02 00:00:00', 0, 603240.3, 603240.3, 603240.3, 'CCo-03'), ('T-008', 5, 'Structural Construction', '2020-03-05 00:00:00', '2020-07-11 00:00:00', 1, 536952.7, 536952.7, 472478.4, 'CCo-04'), ('T-009', 2, 'Monument Foundation', '4/19/2018', '2019-06-06 00:00:00', 0, 225431.1, 245160.3, 225431.1, 'CCo-07

In [8]:
db.get_context()

{'table_info': '\nCREATE TABLE "Billing" (\n\t"BillingId" TEXT, \n\t"ProjectId" INTEGER, \n\t"BillingAmount" REAL, \n\t"BillingDate" TEXT, \n\t"CollectedAmount" REAL, \n\t"DueDate" TEXT\n)\n\n/*\n3 rows from Billing table:\nBillingId\tProjectId\tBillingAmount\tBillingDate\tCollectedAmount\tDueDate\nB-0001\t15\t37238.6\t4/21/2019\t37238.6\t6/25/2019\nB-0002\t37\t376237.5\t9/20/2018\t341287.5\t2019-01-01 00:00:00\nB-0003\t33\t544129.4\t5/19/2022\t544129.4\t2022-03-11 00:00:00\n*/\n\n\nCREATE TABLE "ChangeOrder" (\n\t"ChangeOrderID" TEXT, \n\t"ProjectID" INTEGER, \n\t"Description" TEXT, \n\t"Amount" REAL, \n\t"Date" TEXT, \n\t"Impact_on_Budget" REAL\n)\n\n/*\n3 rows from ChangeOrder table:\nChangeOrderID\tProjectID\tDescription\tAmount\tDate\tImpact_on_Budget\nCO-0001\t3\tAdditional costs due to unforeseen regulatory compliance.\t234070.59\t2020-02-05 00:00:00\t234070.59\nCO-0002\t41\tRevised project scope with no budget impact.\t979801.92\t3/29/2019\t0.0\nCO-0003\t47\tIncorporated cost-e

In [15]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o-mini",temperature=0)

In [55]:
from langchain.chains import create_sql_query_chain
from langchain.agents import create_sql_agent

sql_obj=create_sql_agent(llm=llm ,db=db)
ans=sql_obj.run("give me the employee name with the highest salary")

# chain=create_sql_query_chain(llm,db)
# res=chain.invoke({"question":"how many tasks are there"})


In [56]:
ans

'Ian Garcia'

In [50]:

class SQLAgentTool:
    """
    A tool for interacting with a travel-related SQL database using an LLM (Language Model) to generate and execute SQL queries.

    This tool enables users to ask travel-related questions, which are transformed into SQL queries by a language model.
    The SQL queries are executed on the provided SQLite database, and the results are processed by the language model to
    generate a final answer for the user.

    Attributes:
        sql_agent_llm (ChatOpenAI): An instance of a ChatOpenAI language model used to generate and process SQL queries.
        system_role (str): A system prompt template that guides the language model in answering user questions based on SQL query results.
        db (SQLDatabase): An instance of the SQL database used to execute queries.
        chain (RunnablePassthrough): A chain of operations that creates SQL queries, executes them, and generates a response.

    Methods:
        __init__: Initializes the TravelSQLAgentTool by setting up the language model, SQL database, and query-answering pipeline.
    """

    def __init__(self, llm: str, sqldb_directory: str, llm_temerature: float) -> None:
        """
        Initializes the TravelSQLAgentTool with the necessary configurations.

        Args:
            llm (str): The name of the language model to be used for generating and interpreting SQL queries.
            sqldb_directory (str): The directory path where the SQLite database is stored.
            llm_temerature (float): The temperature setting for the language model, controlling response randomness.
        """
        self.sql_agent_llm = ChatOpenAI(
            model="gpt-4o-mini", temperature=0.0)
        # self.system_role = """Given the following user question, corresponding SQL query, and SQL result, answer the user question.\n
        #     Question: {question}\n
        #     SQL Query: {query}\n
        #     SQL Result: {result}\n
        #     Answer:
        #     """
        self.db = SQLDatabase.from_uri(
            f"sqlite:///{sqldb_directory}")
        print(self.db.get_usable_table_names())

        print("table name printed")

        self.chain=create_sql_agent(llm=self.sql_agent_llm , db=self.db)

        # execute_query = QuerySQLDataBaseTool(db=self.db)
        # write_query = create_sql_query_chain(
        #     self.sql_agent_llm, self.db)
        # answer_prompt = PromptTemplate.from_template(
        #     self.system_role)

        # answer = answer_prompt | self.sql_agent_llm | StrOutputParser()
        # self.chain = (
        #     RunnablePassthrough.assign(query=write_query).assign(
        #         result=itemgetter("query") | execute_query
        #     )
        #     | answer
        # )

In [ ]:
from langchain_core.tools import tool
@tool
def query_sqldb(query: str) -> str:
    """Query SQL Database and access all the information. Input should be a search query."""
    agent = SQLAgentTool(
        llm="gpt-4o-mini",
        sqldb_directory="BIRD (1).db",
        llm_temerature=0.0
    )
    # response = agent.chain.run({"question": query})
    response = agent.chain.run(query)

    return response

In [54]:
query_sqldb("How many tables do I have in the database? and what are their names?")

['Billing', 'ChangeOrder', 'ComplianceIssue', 'Contract', 'CostCodes', 'Employee', 'Expense', 'Invoice', 'Projects', 'Revenue', 'SubContract', 'Task', 'Time_Tracking']
table name printed


'There are 13 tables in the database: Billing, ChangeOrder, ComplianceIssue, Contract, CostCodes, Employee, Expense, Invoice, Projects, Revenue, SubContract, Task, Time_Tracking.'

In [35]:
print(ans)

There are 12 tables in the database: Billing, ChangeOrder, ComplianceIssue, Contract, CostCodes, Employee, Expense, Invoice, Projects, Revenue, SubContract, Task, Time_Tracking.


In [23]:
chain.get_prompts()[0].template

'You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use date(\'now\') function to get the current date, if the question involves "today".\n\nUse the following format:\n\nQuestion: Question here\nSQLQuery: SQL Query to run

In [24]:
'You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use date(\'now\') function to get the current date, if the question involves "today".\n\nUse the following format:\n\nQuestion: Question here\nSQLQuery: SQL Query to run\nSQLResult: Result of the SQLQuery\nAnswer: Final answer here\n\nOnly use the following tables:\n{table_info}\n\nQuestion: {input}'

'You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use date(\'now\') function to get the current date, if the question involves "today".\n\nUse the following format:\n\nQuestion: Question here\nSQLQuery: SQL Query to run

In [30]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
write_query=create_sql_query_chain(llm,db)
execute_query=QuerySQLDataBaseTool(db=db)

chain= write_query | execute_query
chain.invoke({"question":"how many tasks are there"})

'Error: (sqlite3.OperationalError) near "SQLQuery": syntax error\n[SQL: SQLQuery: SELECT COUNT("TaskID") AS "TotalTasks" FROM "Task";]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'

In [31]:

from pyprojroot import here
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
system_role = """Given the following user question, corresponding SQL query, and SQL result, answer the user question.\n
    Question: {question}\n
    SQL Query: {query}\n
    SQL Result: {result}\n
    Answer:
    """

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(
    llm, db)
answer_prompt = PromptTemplate.from_template(
    system_role)
answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

In [33]:
message = "How many tables do I have in the database? and what are their names?"
response = chain.invoke({"question": message})
response

'It seems there was an error in the SQL query you attempted to run. The correct SQL query to find out how many tables you have in the database and their names is:\n\n```sql\nSELECT "name" FROM sqlite_master WHERE "type" = \'table\';\n```\n\nHowever, since the query resulted in an error, it indicates that there might have been an issue with how the query was executed or formatted. \n\nTo answer your question, you need to run the corrected SQL query without any additional text (like "SQLQuery:") before it. Once you run the correct query, you will get a list of table names in your database. \n\nIf the query executes successfully, you can count the number of rows returned to determine how many tables you have, and the names of the tables will be listed in the result.'